In [1]:
# import SAGA code

import SAGA
from SAGA import ObjectCuts as C
from SAGA.utils import get_sdss_bands

print('SAGA code version', SAGA.__version__)

SAGA code version 0.2.1


In [2]:
# import other modules

from collections import defaultdict
import numpy as np
from astropy.table import Table
from easyquery import Query

In [3]:
# initialize SAGA objects (Database, HostCatalog, ObjectCatalog)
# NOTE: change the path of `root_dir` to your SAGA dropbox path

saga_database = SAGA.Database(root_dir='/home/yymao/Dropbox/Academia/Collaborations/SAGA')
saga_host_catalog = SAGA.HostCatalog(saga_database)
saga_object_catalog = SAGA.ObjectCatalog(saga_database)

In [4]:
# load flag-0 host catalog

hosts = saga_host_catalog.load('no_flags')

In [5]:
# load paper1 base catalogs as a list

columns = 'OBJID HOST_NSAID SATS ZQUALITY'.split()
columns.extend('{}_mag'.format(b) for b in get_sdss_bands())
columns.extend('{}_mag'.format(b) for b in get_sdss_bands())

cuts = C.is_clean & C.is_galaxy & C.fibermag_r_cut & C.sat_rcut

objects_paper1 = saga_object_catalog.load(hosts='paper1', cuts=cuts, columns=columns, return_as='list')

ValueError: Duplicate column names

In [ ]:
# construct Table 1

host_info_cols = ['NSAID', 'RA', 'Dec', 'distance', 'M_r', 'M_K']

sat_Mr_compl = Query('M_r < -12.3')
counting_cols = [
    ('N_sat', C.is_sat & sat_Mr_compl),
    ('N_sat_below_compl', C.is_sat & ~sat_Mr_compl),
    ('N_total', C.faint_end_limit),
    ('N_gri_spec', C.faint_end_limit & C.gri_cut & C.has_spec),
    ('N_gri', C.faint_end_limit & C.gri_cut),
    ('N_ugri_spec', C.faint_end_limit & C.ugri_cut & C.has_spec),
    ('N_ugri', C.faint_end_limit & C.ugri_cut),
]

data = defaultdict(list)

for t in objects_paper1:
    host_id = t['HOST_NSAID'][0]
    host_info = Query('NSAID == {}'.format(host_id)).filter(hosts)[0]
    t['M_r'] = t['r_mag'] - (5.0*(np.log10(host_info['distance'])+6.0)-5.0)
    
    data['SAGA_name'].append(saga_host_catalog.id_to_name(host_id))

    for col in host_info_cols:
        data[col].append(host_info[col])
        
    for col, cut in counting_cols:
        data[col].append(cut.count(t))
        
Table(data)